# TensorFlow 模型建立与训练 

本章介绍如何使用 TensorFlow 快速搭建动态模型。

- 模型的构建： `tf.keras.Model` 和 `tf.keras.layers`

- 模型的损失函数： `tf.keras.losses`

- 模型的优化器： `tf.keras.optimizer`

- 模型的评估： `tf.keras.metrics`/

## 模型（Model）与层（Layer）

在 TensorFlow 中，推荐使用 Keras（ `tf.keras` ）构建模型。Keras 是一个广为流行的高级神经网络 API，简单、快速而不失灵活性，现已得到 TensorFlow 的官方内置和全面支持。

Keras 有两个重要的概念： **模型（Model）** 和 **层（Layer）** 。层将各种计算流程和变量进行了封装（例如基本的全连接层，CNN 的卷积层、池化层等），而模型则将各种层进行组织和连接，并封装成一个整体，描述了如何将输入数据通过各种层以及运算而得到输出。在需要模型调用的时候，使用 `y_pred = model(X)` 的形式即可。Keras 在 `tf.keras.layers` 下内置了深度学习中大量常用的的预定义层，同时也允许我们自定义层。

Keras 模型以类的形式呈现，我们可以通过继承 `tf.keras.Model` 这个 Python 类来定义自己的模型。在继承类中，我们需要重写 `__init__()` （构造函数，初始化）和 `call(input)` （模型调用）两个方法，同时也可以根据需要增加自定义的方法。

In [1]:
import tensorflow as tf
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()     # Python 2 下使用 super(MyModel, self).__init__()
        # 此处添加初始化代码（包含 call 方法中会用到的层），例如
        # layer1 = tf.keras.layers.BuiltInLayer(...)
        # layer2 = MyCustomLayer(...)

    def call(self, input):
        # 此处添加模型调用的代码（处理输入并返回输出），例如
        # x = layer1(input)
        # output = layer2(x)
        return output

    # 还可以添加自定义的方法

[![../../_images/model.png](https://tf.wiki/_images/model.png)](https://tf.wiki/_images/model.png)

Keras 模型类定义示意图 

继承 `tf.keras.Model` 后，我们同时可以使用父类的若干方法和属性，例如在实例化类 `model = Model()` 后，可以通过 `model.variables` 这一属性直接获得模型中的所有变量，免去我们一个个显式指定变量的麻烦。

上一章中简单的线性模型 `y_pred = a * X + b` ，我们可以通过模型类的方式编写如下：

In [2]:
x = tf.constant([[1., 2., 3.],[4., 5., 6.]])
y = tf.constant([10.0, 20.0])

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.Dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer,
            bias_initializer=tf.zeros_initializer,
        )
    def call(self, input):
        out = self.Dense(input)
        return out

# 模型训练
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = tf.reduce_sum(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars = zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear/dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[nan],
       [nan],
       [nan]], dtype=float32)>, <tf.Variable 'linear/dense/bias:0' shape=(1,) dtype=float32, numpy=array([nan], dtype=float32)>]


这里，我们没有显式地声明 `a` 和 `b` 两个变量并写出 `y_pred = a * X + b` 这一线性变换，而是建立了一个继承了 `tf.keras.Model` 的模型类 `Linear` 。这个类在初始化部分实例化了一个 **全连接层** （ `tf.keras.layers.Dense` ），并在 call 方法中对这个层进行调用，实现了线性变换的计算。如果需要显式地声明自己的变量并使用变量进行自定义运算，或者希望了解 Keras 层的内部原理，请参考 [自定义层](https://tf.wiki/zh/basic/models.html#custom-layer)。

Keras 的全连接层：线性变换 + 激活函数

[全连接层](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) （Fully-connected Layer，`tf.keras.layers.Dense` ）是 Keras 中最基础和常用的层之一，对输入矩阵 ![A](https://tf.wiki/_images/math/211284f68205c3e66773eaf026f32a0acdd3dfb3.png) 进行 ![f(AW + b)](https://tf.wiki/_images/math/c324a7004aff2d31917fb70e1311c4f43449a14c.png) 的线性变换 + 激活函数操作。如果不指定激活函数，即是纯粹的线性变换 ![AW + b](https://tf.wiki/_images/math/7195d0ad48bfa4fb60a47b82eb81b21d21ef9d9f.png)。具体而言，给定输入张量 `input = [batch_size, input_dim]` ，该层对输入张量首先进行 `tf.matmul(input, kernel) + bias` 的线性变换（ `kernel` 和 `bias` 是层中可训练的变量），然后对线性变换后张量的每个元素通过激活函数 `activation` ，从而输出形状为 `[batch_size, units]` 的二维张量。

[![../../_images/dense.png](https://tf.wiki/_images/dense.png)](https://tf.wiki/_images/dense.png)

其包含的主要参数如下：

- `units` ：输出张量的维度；
- `activation` ：激活函数，对应于 ![f(AW + b)](https://tf.wiki/_images/math/c324a7004aff2d31917fb70e1311c4f43449a14c.png) 中的 ![f](https://tf.wiki/_images/math/5b7752c757e0b691a80ab8227eadb8a8389dc58a.png) ，默认为无激活函数（ `a(x) = x` ）。常用的激活函数包括 `tf.nn.relu` 、 `tf.nn.tanh` 和 `tf.nn.sigmoid` ；
- `use_bias` ：是否加入偏置向量 `bias` ，即 ![f(AW + b)](https://tf.wiki/_images/math/c324a7004aff2d31917fb70e1311c4f43449a14c.png) 中的 ![b](https://tf.wiki/_images/math/68c7c8c65602677ab56cf7fd88002023f0edc575.png)。默认为 `True` ；
- `kernel_initializer` 、 `bias_initializer` ：权重矩阵 `kernel` 和偏置向量 `bias` 两个变量的初始化器。默认为 `tf.glorot_uniform_initializer` [1](https://tf.wiki/zh/basic/models.html#glorot) 。设置为 `tf.zeros_initializer` 表示将两个变量均初始化为全 0；

该层包含权重矩阵 `kernel = [input_dim, units]` 和偏置向量 `bias = [units]` [2](https://tf.wiki/zh/basic/models.html#broadcast) 两个可训练变量，对应于 ![f(AW + b)](https://tf.wiki/_images/math/c324a7004aff2d31917fb70e1311c4f43449a14c.png) 中的 ![W](https://tf.wiki/_images/math/1fbee781f84569077719a167b64e12064360fac1.png) 和 ![b](https://tf.wiki/_images/math/68c7c8c65602677ab56cf7fd88002023f0edc575.png)。

这里着重从数学矩阵运算和线性变换的角度描述了全连接层。基于神经元建模的描述可参考 [后文介绍](https://tf.wiki/zh/basic/models.html#neuron) 。

- [1](https://tf.wiki/zh/basic/models.html#id3)

  Keras 中的很多层都默认使用 `tf.glorot_uniform_initializer` 初始化变量，关于该初始化器可参考 https://www.tensorflow.org/api_docs/python/tf/glorot_uniform_initializer 。

- [2](https://tf.wiki/zh/basic/models.html#id4)

  你可能会注意到， `tf.matmul(input, kernel)` 的结果是一个形状为 `[batch_size, units]` 的二维矩阵，这个二维矩阵要如何与形状为 `[units]` 的一维偏置向量 `bias` 相加呢？事实上，这里是 TensorFlow 的 Broadcasting 机制在起作用，该加法运算相当于将二维矩阵的每一行加上了 `Bias` 。Broadcasting 机制的具体介绍可见 https://www.tensorflow.org/xla/broadcasting 。

为什么模型类是重载 `call()` 方法而不是 `__call__()` 方法？

在 Python 中，对类的实例 `myClass` 进行形如 `myClass()` 的调用等价于 `myClass.__call__()` （具体请见本章初 “前置知识” 的 `__call__()` 部分）。那么看起来，为了使用 `y_pred = model(X)` 的形式调用模型类，应该重写 `__call__()` 方法才对呀？原因是 Keras 在模型调用的前后还需要有一些自己的内部操作，所以暴露出一个专门用于重载的 `call()` 方法。 `tf.keras.Model` 这一父类已经包含 `__call__()` 的定义。 `__call__()` 中主要调用了 `call()` 方法，同时还需要在进行一些 keras 的内部操作。这里，我们通过继承 `tf.keras.Model` 并重载 `call()` 方法，即可在保持 keras 结构的同时加入模型调用的代码。

## 基础示例：多层感知机（MLP）

我们从编写一个最简单的 [多层感知机](https://zh.wikipedia.org/wiki/多层感知器) （Multilayer Perceptron, MLP），或者说 “多层全连接神经网络” 开始，介绍 TensorFlow 的模型编写方式。在这一部分，我们依次进行以下步骤：

- 使用 `tf.keras.datasets` 获得数据集并预处理
- 使用 `tf.keras.Model` 和 `tf.keras.layers` 构建模型
- 构建模型训练流程，使用 `tf.keras.losses` 计算损失函数，并使用 `tf.keras.optimizer` 优化模型
- 构建模型评估流程，使用 `tf.keras.metrics` 计算评估指标

基础知识和原理

- UFLDL 教程 [Multi-Layer Neural Network](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/) 一节；
- 斯坦福课程 [CS231n: Convolutional Neural Networks for Visual Recognition](https://cs231n.github.io/) 中的 “Neural Networks Part 1 ~ 3” 部分。

这里，我们使用多层感知机完成 MNIST 手写体数字图片数据集 [[LeCun1998\]](https://tf.wiki/zh/basic/models.html#lecun1998) 的分类任务。

![../../_images/mnist_0-9.png](https://tf.wiki/_images/mnist_0-9.png)

MNIST 手写体数字图片示例 

### 数据获取及预处理： `tf.keras.datasets`

先进行预备工作，实现一个简单的 `MNISTLoader` 类来读取 MNIST 数据集数据。这里使用了 `tf.keras.datasets` 快速载入 MNIST 数据集。

In [16]:
# 数据获取及预处理
import numpy as np
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

`mnist = tf.keras.datasets.mnist` 将从网络上自动下载 MNIST 数据集并加载。如果运行时出现网络连接错误，可以从 https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz 或 https://s3.amazonaws.com/img-datasets/mnist.npz 下载 MNIST 数据集 `mnist.npz` 文件，并放置于用户目录的 `.keras/dataset` 目录下（Windows 下用户目录为 `C:\Users\用户名` ，Linux 下用户目录为 `/home/用户名` ）。

TensorFlow 的图像数据表示

在 TensorFlow 中，图像数据集的一种典型表示是 `[图像数目，长，宽，色彩通道数]` 的四维张量。在上面的 `DataLoader` 类中， `self.train_data` 和 `self.test_data` 分别载入了 60,000 和 10,000 张大小为 `28*28` 的手写体数字图片。由于这里读入的是灰度图片，色彩通道数为 1（彩色 RGB 图像色彩通道数为 3），所以我们使用 `np.expand_dims()` 函数为图像数据手动在最后添加一维通道。

### 模型的构建： `tf.keras.Model` 和 `tf.keras.layers`

多层感知机的模型类实现与上面的线性模型类似，使用 `tf.keras.Model` 和 `tf.keras.layers` 构建，所不同的地方在于层数增加了（顾名思义，“多层” 感知机），以及引入了非线性激活函数（这里使用了 [ReLU 函数](https://zh.wikipedia.org/wiki/线性整流函数) ， 即下方的 `activation=tf.nn.relu` ）。该模型输入一个向量（比如这里是拉直的 `1×784` 手写体数字图片），输出 10 维的向量，分别代表这张图片属于 0 到 9 的概率。

In [26]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

softmax 函数

这里，因为我们希望输出 “输入图片分别属于 0 到 9 的概率”，也就是一个 10 维的离散概率分布，所以我们希望这个 10 维向量至少满足两个条件：

- 该向量中的每个元素均在 ![[0, 1]](https://tf.wiki/_images/math/8027137b3073a7f5ca4e45ba2d030dcff154eca4.png) 之间；
- 该向量的所有元素之和为 1。

为了使得模型的输出能始终满足这两个条件，我们使用 [Softmax 函数](https://zh.wikipedia.org/wiki/Softmax函数) （归一化指数函数， `tf.nn.softmax` ）对模型的原始输出进行归一化。其形式为 ![\sigma(\mathbf{z})_j = \frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}](https://tf.wiki/_images/math/7d714874b555007ada90b5315e5fa2ffa0e5e2ee.png) 。不仅如此，softmax 函数能够凸显原始向量中最大的值，并抑制远低于最大值的其他分量，这也是该函数被称作 softmax 函数的原因（即平滑化的 argmax 函数）。

[![../../_images/mlp.png](https://tf.wiki/_images/mlp.png)](https://tf.wiki/_images/mlp.png)

MLP 模型示意图 


### 模型的训练： `tf.keras.losses` 和 `tf.keras.optimizer`

定义一些模型超参数：

In [18]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [19]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

然后迭代进行以下步骤：

- 从 DataLoader 中随机取一批训练数据；
- 将这批数据送入模型，计算出模型的预测值；
- 将模型预测值与真实值进行比较，计算损失函数（loss）。这里使用 `tf.keras.losses` 中的交叉熵函数作为损失函数；
- 计算损失函数关于模型变量的导数；
- 将求出的导数值传入优化器，使用优化器的 `apply_gradients` 方法更新模型参数以最小化损失函数（优化器的详细使用方法见 [前章](https://tf.wiki/en/basic/basic.html#optimizer) ）。

具体代码实现如下：

In [21]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

11765
batch 5231: loss 0.020297
batch 5232: loss 0.009016
batch 5233: loss 0.029058
batch 5234: loss 0.020301
batch 5235: loss 0.065630
batch 5236: loss 0.117782
batch 5237: loss 0.023475
batch 5238: loss 0.017457
batch 5239: loss 0.031909
batch 5240: loss 0.090876
batch 5241: loss 0.029181
batch 5242: loss 0.052658
batch 5243: loss 0.045131
batch 5244: loss 0.020806
batch 5245: loss 0.072734
batch 5246: loss 0.035175
batch 5247: loss 0.021386
batch 5248: loss 0.018279
batch 5249: loss 0.013514
batch 5250: loss 0.028197
batch 5251: loss 0.020028
batch 5252: loss 0.011727
batch 5253: loss 0.129560
batch 5254: loss 0.027805
batch 5255: loss 0.008097
batch 5256: loss 0.141144
batch 5257: loss 0.015935
batch 5258: loss 0.031550
batch 5259: loss 0.041442
batch 5260: loss 0.025335
batch 5261: loss 0.011035
batch 5262: loss 0.088995
batch 5263: loss 0.021100
batch 5264: loss 0.147815
batch 5265: loss 0.017561
batch 5266: loss 0.019981
batch 5267: loss 0.005434
batch 5268: loss 0.064148
batch 

交叉熵（cross entropy）与 `tf.keras.losses`

你或许注意到了，在这里，我们没有显式地写出一个损失函数，而是使用了 `tf.keras.losses` 中的 `sparse_categorical_crossentropy` （交叉熵）函数，将模型的预测值 `y_pred` 与真实的标签值 `y` 作为函数参数传入，由 Keras 帮助我们计算损失函数的值。

交叉熵作为损失函数，在分类问题中被广泛应用。其离散形式为 ![H(y, \hat{y}) = -\sum_{i=1}^{n}y_i \log(\hat{y_i})](https://tf.wiki/_images/math/63ad0688c80b4c83b2a6e0a542b741ed8f9ff79f.png) ，其中 ![y](https://tf.wiki/_images/math/1b5e577d6216dca3af7d87aa122a0b9b360d6cb3.png) 为真实概率分布， ![\hat{y}](https://tf.wiki/_images/math/1257829df10bd602e03553570cadfe2328fd1d91.png) 为预测概率分布， ![n](https://tf.wiki/_images/math/5a939c5280da7202ca4531f175a7780ad5e1f80a.png) 为分类任务的类别个数。预测概率分布与真实分布越接近，则交叉熵的值越小，反之则越大。更具体的介绍及其在机器学习中的应用可参考 [这篇博客文章](https://blog.csdn.net/tsyccnh/article/details/79163834) 。

在 `tf.keras` 中，有两个交叉熵相关的损失函数 `tf.keras.losses.categorical_crossentropy` 和 `tf.keras.losses.sparse_categorical_crossentropy` 。其中 sparse 的含义是，真实的标签值 `y_true` 可以直接传入 int 类型的标签类别。具体而言：

```
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
```

与

```
loss = tf.keras.losses.categorical_crossentropy(
    y_true=tf.one_hot(y, depth=tf.shape(y_pred)[-1]),
    y_pred=y_pred
)
```

In [23]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.978000
